In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c cse-251-b-2025

 97% 970M/0.98G [00:09<00:00, 195MB/s]
100% 0.98G/0.98G [00:09<00:00, 111MB/s]


In [ ]:
!unzip cse-251-b-2025.zip -d data/

Archive:  cse-251-b-2025.zip
  inflating: data/test_input.npz     
  inflating: data/train.npz          


In [1]:
# load data
import numpy as np
train_file = np.load('argoverse_data/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)
test_file = np.load('argoverse_data/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List, Optional
import random

class ArgoverseAugmentor:
    """
    Data augmentation class for Argoverse trajectory data.
    Supports rotations, translations, and horizontal/vertical flips.
    """
    
    def __init__(self, seed: Optional[int] = None):
        """
        Initialize the augmentor.
        
        Args:
            seed: Random seed for reproducible augmentations
        """
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
    
    def rotate_trajectory(self, trajectory: np.ndarray, angle: float) -> np.ndarray:
        """
        Rotate trajectory by given angle around the origin.
        Assumes first 2 dimensions are (x, y) coordinates.
        
        Args:
            trajectory: Shape (N, F) array where F >= 2, first 2 cols are (x, y)
            angle: Rotation angle in radians
            
        Returns:
            Rotated trajectory of same shape
        """
        if trajectory.shape[-1] < 2:
            raise ValueError("Trajectory must have at least 2 features (x, y)")
            
        cos_a, sin_a = np.cos(angle), np.sin(angle)
        rotation_matrix = np.array([[cos_a, -sin_a],
                                   [sin_a, cos_a]])
        
        # Only rotate x, y coordinates (first 2 columns)
        rotated_trajectory = trajectory.copy()
        xy_coords = trajectory[:, :2]  # Extract x, y coordinates
        rotated_xy = xy_coords @ rotation_matrix.T
        rotated_trajectory[:, :2] = rotated_xy  # Update only x, y
        
        return rotated_trajectory
    
    def translate_trajectory(self, trajectory: np.ndarray, 
                           dx: float, dy: float) -> np.ndarray:
        """
        Translate trajectory by given offsets.
        Only translates x, y coordinates (first 2 features).
        
        Args:
            trajectory: Shape (N, F) array where F >= 2, first 2 cols are (x, y)
            dx: Translation in x direction
            dy: Translation in y direction
            
        Returns:
            Translated trajectory of same shape
        """
        if trajectory.shape[-1] < 2:
            raise ValueError("Trajectory must have at least 2 features (x, y)")
            
        translated_trajectory = trajectory.copy()
        translated_trajectory[:, 0] += dx  # Translate x
        translated_trajectory[:, 1] += dy  # Translate y
        
        return translated_trajectory
    
    def flip_horizontal(self, trajectory: np.ndarray) -> np.ndarray:
        """
        Flip trajectory horizontally (mirror across y-axis).
        Only affects x coordinate (first feature).
        
        Args:
            trajectory: Shape (N, F) array where F >= 2, first 2 cols are (x, y)
            
        Returns:
            Horizontally flipped trajectory
        """
        if trajectory.shape[-1] < 2:
            raise ValueError("Trajectory must have at least 2 features (x, y)")
            
        flipped = trajectory.copy()
        flipped[:, 0] = -flipped[:, 0]  # Negate x coordinates
        return flipped
    
    def flip_vertical(self, trajectory: np.ndarray) -> np.ndarray:
        """
        Flip trajectory vertically (mirror across x-axis).
        Only affects y coordinate (second feature).
        
        Args:
            trajectory: Shape (N, F) array where F >= 2, first 2 cols are (x, y)
            
        Returns:
            Vertically flipped trajectory
        """
        if trajectory.shape[-1] < 2:
            raise ValueError("Trajectory must have at least 2 features (x, y)")
            
        flipped = trajectory.copy()
        flipped[:, 1] = -flipped[:, 1]  # Negate y coordinates
        return flipped
    
    def augment_single_sample(self, sample: np.ndarray, 
                            rotation_range: float = np.pi/4,
                            translation_range: float = 5.0,
                            flip_prob: float = 0.5) -> np.ndarray:
        """
        Apply random augmentations to a single sample.
        
        Args:
            sample: Trajectory data - shape depends on format:
                   - (seq_len, features) for single agent
                   - (seq_len, num_agents, features) for multi-agent
            rotation_range: Maximum rotation angle in radians
            translation_range: Maximum translation distance
            flip_prob: Probability of applying flips
            
        Returns:
            Augmented sample of same shape
        """
        augmented = sample.copy()
        
        # Random rotation
        angle = np.random.uniform(-rotation_range, rotation_range)
        
        # Random translation
        dx = np.random.uniform(-translation_range, translation_range)
        dy = np.random.uniform(-translation_range, translation_range)
        
        # Handle different data formats
        if len(sample.shape) == 2:
            # Single agent: (seq_len, features)
            trajectory = sample
            if not np.all(trajectory == 0):  # Skip if all zeros
                trajectory = self.rotate_trajectory(trajectory, angle)
                trajectory = self.translate_trajectory(trajectory, dx, dy)
                
                if np.random.random() < flip_prob:
                    trajectory = self.flip_horizontal(trajectory)
                
                if np.random.random() < flip_prob:
                    trajectory = self.flip_vertical(trajectory)
                    
                augmented = trajectory
                
        elif len(sample.shape) == 3:
            # Multi-agent: (seq_len, num_agents, features)
            for agent_idx in range(sample.shape[1]):
                trajectory = sample[:, agent_idx, :]
                
                # Skip if trajectory is all zeros (padding)
                if np.all(trajectory == 0):
                    continue
                    
                # Apply rotation and translation
                trajectory = self.rotate_trajectory(trajectory, angle)
                trajectory = self.translate_trajectory(trajectory, dx, dy)
                
                # Apply flips with probability
                if np.random.random() < flip_prob:
                    trajectory = self.flip_horizontal(trajectory)
                
                if np.random.random() < flip_prob:
                    trajectory = self.flip_vertical(trajectory)
                    
                augmented[:, agent_idx, :] = trajectory
        else:
            raise ValueError(f"Unsupported sample shape: {sample.shape}")
        
        return augmented
    
    def augment_dataset(self, data: np.ndarray, 
                       num_augmentations: int = 1,
                       rotation_range: float = np.pi/4,
                       translation_range: float = 5.0,
                       flip_prob: float = 0.5,
                       return_original: bool = True) -> np.ndarray:
        """
        Augment entire dataset.
        
        Args:
            data: Shape (N, seq_len, num_agents, 2) dataset
            num_augmentations: Number of augmented versions per sample
            rotation_range: Maximum rotation angle in radians
            translation_range: Maximum translation distance
            flip_prob: Probability of applying flips
            return_original: Whether to include original data in output
            
        Returns:
            Augmented dataset with shape (N*(1+num_augmentations), seq_len, num_agents, 2)
        """
        original_size = data.shape[0]
        augmented_samples = []
        
        # Add original data if requested
        if return_original:
            augmented_samples.append(data)
        
        # Generate augmentations
        for aug_idx in range(num_augmentations):
            print(f"Generating augmentation {aug_idx + 1}/{num_augmentations}...")
            
            augmented_batch = np.zeros_like(data)
            for i in range(original_size):
                augmented_batch[i] = self.augment_single_sample(
                    data[i], rotation_range, translation_range, flip_prob
                )
            
            augmented_samples.append(augmented_batch)
        
        return np.concatenate(augmented_samples, axis=0)
    
    def visualize_augmentations(self, original_sample: np.ndarray, 
                              num_examples: int = 4,
                              agent_idx: int = 0) -> None:
        """
        Visualize original and augmented trajectories for comparison.
        
        Args:
            original_sample: Single sample to augment and visualize
            num_examples: Number of augmented examples to show
            agent_idx: Which agent's trajectory to visualize (for multi-agent data)
        """
        fig, axes = plt.subplots(1, num_examples + 1, figsize=(15, 3))
        
        # Extract trajectory based on data format
        if len(original_sample.shape) == 2:
            # Single agent data
            orig_traj = original_sample[:, :2]  # Use only x, y coordinates
        elif len(original_sample.shape) == 3:
            # Multi-agent data
            orig_traj = original_sample[:, agent_idx, :2]  # Use only x, y coordinates
        else:
            raise ValueError(f"Unsupported sample shape: {original_sample.shape}")
        
        # Plot original
        axes[0].plot(orig_traj[:, 0], orig_traj[:, 1], 'b-o', markersize=3)
        axes[0].set_title('Original')
        axes[0].grid(True)
        axes[0].axis('equal')
        
        # Plot augmented versions
        for i in range(num_examples):
            aug_sample = self.augment_single_sample(original_sample)
            
            # Extract augmented trajectory
            if len(aug_sample.shape) == 2:
                aug_traj = aug_sample[:, :2]
            else:
                aug_traj = aug_sample[:, agent_idx, :2]
            
            axes[i + 1].plot(aug_traj[:, 0], aug_traj[:, 1], 'r-o', markersize=3)
            axes[i + 1].set_title(f'Augmented {i + 1}')
            axes[i + 1].grid(True)
            axes[i + 1].axis('equal')
        
        plt.tight_layout()
        plt.show()


In [5]:
augmentor = ArgoverseAugmentor(seed=42)

# Example 1: Augment training data with 2 additional versions per sample
print("\nAugmenting training data...")
augmented_train = augmentor.augment_dataset(
    train_data,
    num_augmentations=0,  # Create 2 augmented versions per sample
    rotation_range=np.pi/6,  # ±30 degrees
    translation_range=3.0,   # ±3 units
    flip_prob=0.3,           # 30% chance of flips
    return_original=True     # Include original data
)

print(f"Augmented train_data shape: {augmented_train.shape}")
print(f"Data increased by factor of: {augmented_train.shape[0] / train_data.shape[0]:.1f}")


Augmenting training data...
Augmented train_data shape: (10000, 50, 110, 6)
Data increased by factor of: 1.0


In [6]:
train_data = augmented_train

In [7]:
# preprocess
def preprocess_data(data):
    """
    Removes padded agents (agents with all zero values across time steps).

    Args:
        data (numpy.ndarray): Shape (scenarios, agents, time_steps, dimensions)

    Returns:
        numpy.ndarray: Filtered dataset without padded agents.
    """
    scenarios, agents, time_steps, dimensions = data.shape
    processed_data = []

    for i in range(scenarios):
        scenario_data = data[i]  # Shape (agents, time_steps, dimensions)

        # Identify non-padded agents (at least one nonzero value across all time steps)
        valid_agents = np.any(scenario_data != 0, axis=(1, 2))  # Shape (agents,)

        # Filter out only the valid agents
        filtered_agents = scenario_data[valid_agents]  # Shape (valid_agents, time_steps, dimensions)

        processed_data.append(filtered_agents)

    return processed_data  # List of variable-length arrays per scenario

train_data_processed = preprocess_data(train_data)
test_data_processed = preprocess_data(test_data)

# Print results
print(f"Original Train Data Shape: {train_data.shape}")
print(f"Processed Train Data Length: {len(train_data_processed)} (variable agents per scenario)")

print(f"Original Test Data Shape: {test_data.shape}")
print(f"Processed Test Data Length: {len(test_data_processed)} (variable agents per scenario)")

Original Train Data Shape: (10000, 50, 110, 6)
Processed Train Data Length: 10000 (variable agents per scenario)
Original Test Data Shape: (2100, 50, 50, 6)
Processed Test Data Length: 2100 (variable agents per scenario)


In [8]:
# some kind of missing trajectory handling?

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed, Dropout, Bidirectional, LayerNormalization, Flatten, Concatenate

2025-05-30 00:53:46.015764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-30 00:53:46.015829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-30 00:53:46.017484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 00:53:46.027180: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
import tensorflow as tf

def angle_change_loss(y_pred):
    """
    Penalize large changes in direction between consecutive deltas.

    Args:
        y_pred: Tensor of shape (batch_size, Tpred, 2)

    Returns:
        Scalar loss penalizing angle differences
    """
    # Normalize deltas to unit vectors
    delta_unit = tf.math.l2_normalize(y_pred, axis=-1)  # shape: (B, T, 2)

    # Compute cosine similarity between consecutive deltas
    dot_products = tf.reduce_sum(delta_unit[:, 1:, :] * delta_unit[:, :-1, :], axis=-1)  # shape: (B, T-1)

    # Clamp for numerical stability (to avoid NaNs in arccos)
    dot_products = tf.clip_by_value(dot_products, -1.0, 1.0)

    # Compute angle in radians between -1 and 1 (cos⁻¹)
    angle_diff = tf.acos(dot_products)  # shape: (B, T-1)

    # Mean angle difference per sequence
    return tf.reduce_mean(angle_diff)

def combined_loss(y_true, y_pred):
    # Check if there are NaN values in y_true or y_pred
    nan_check = tf.reduce_any(tf.math.is_nan(y_true)) | tf.reduce_any(tf.math.is_nan(y_pred))

    # Use tf.cond to perform the check
    def return_nan_loss():
        return tf.constant(float('nan'))

    def calculate_loss():
        mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
        angle_loss = angle_change_loss(y_pred)
        return mse_loss + 0.5 * angle_loss

    return tf.cond(nan_check, return_nan_loss, calculate_loss)




In [11]:
def create_lstm_encoder_decoder(input_dim, output_dim, timesteps_in, timesteps_out,
                                max_stationary=30, lstm_units=512, num_layers=3, loss_fn='mse', lr=0.001):
    # Two inputs
    trajectory_input = Input(shape=(timesteps_in, input_dim))
    context_input = Input(shape=(max_stationary, 2))
    
    # Encoder for trajectory
    x = trajectory_input
    for _ in range(num_layers):
        x = LSTM(lstm_units, return_sequences=True)(x)
        #x = LayerNormalization()(x)
        # x = Dropout(0.2)(x)
    encoded = LSTM(lstm_units)(x)  # Final encoder output
    
    # Process stationary context
    context_processed = Flatten()(context_input)
    context_processed = Dense(64, activation='relu')(context_processed)
    context_processed = Dense(32, activation='relu')(context_processed)
    
    # Combine encoded trajectory with context
    combined = Concatenate()([encoded, context_processed])
    combined = Dense(lstm_units, activation='relu')(combined)  # Project back to lstm_units size
    
    # Decoder
    x = RepeatVector(timesteps_out)(combined)
    for _ in range(num_layers):
        x = LSTM(lstm_units, return_sequences=True)(x)
        #x = LayerNormalization()(x)
        # x = Dropout(0.2)(x)
    
    x = TimeDistributed(Dense(128, activation='relu'))(x)
    x = TimeDistributed(Dense(64, activation='relu'))(x)
    outputs = TimeDistributed(Dense(output_dim))(x)
    
    model = Model([trajectory_input, context_input], outputs)  # Multiple inputs
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse', metrics=['mae'])
    
    return model

In [12]:
from keras.src.callbacks import LearningRateScheduler, EarlyStopping, Callback
from keras.src.optimizers import Adam
from keras import Model
import numpy as np


def exponential_decay_schedule(epoch, lr):
    decay_rate = 0.9
    decay_steps = 5
    if epoch % decay_steps == 0 and epoch:
        print('Learning rate update:', lr * decay_rate)
        return lr * decay_rate
    return lr

def is_stationary(trajectory, threshold=0.1):
    """Check if agent is stationary based on position variance"""
    positions = trajectory[:, :2]  # x, y coordinates
    return np.all(np.var(positions, axis=0) < threshold)

# Custom callback to monitor LR and stop training
class LRThresholdCallback(Callback):
    def __init__(self, threshold=9e-5):
        super().__init__()
        self.threshold = threshold
        self.should_stop = False

    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate.numpy())
        if lr < self.threshold:
            print(f"\nLearning rate {lr:.6f} < threshold {self.threshold}, moving to Phase 2.")
            self.model.stop_training = True

def train_model(train_data, batch_size=32, validation_split=0.2, Tobs=50, Tpred=60):
    n_scenarios = train_data.shape[0]
    X_train_raw = []
    y_train_deltas = []
    stationary_context = []

    for i in range(n_scenarios):
        # Process ego vehicle as before
        ego_data = train_data[i, 0, :, :]
        if np.all(ego_data == 0):
            continue
        
        observed = ego_data[:Tobs]
        future = ego_data[Tobs:Tobs+Tpred, :2]
        last_obs_pos = observed[-1, :2]
        
        if np.any(np.all(observed == 0, axis=1)) or np.any(np.all(future == 0, axis=1)):
            continue
        
        # Extract stationary agents for this scenario
        scenario_stationary = []
        for vehicle_idx in range(1, train_data.shape[1]):  # Skip ego
            vehicle_data = train_data[i, vehicle_idx, :, :]
            
            if np.all(vehicle_data == 0):
                continue
                
            # Check if agent is present and stationary during observation window
            obs_segment = vehicle_data[:Tobs]
            if not np.any(np.all(obs_segment == 0, axis=1)):
                # Store relative position to ego
                relative_pos = obs_segment[-1, :2] - last_obs_pos
                scenario_stationary.append(relative_pos)
        
        # Pad or limit number of stationary agents
        max_stationary = 30  # Adjust as needed
        if len(scenario_stationary) > max_stationary:
            scenario_stationary = scenario_stationary[:max_stationary]
        else:
            # Pad with zeros
            while len(scenario_stationary) < max_stationary:
                scenario_stationary.append([0.0, 0.0])
        
        delta = np.diff(np.vstack([last_obs_pos, future]), axis=0)
        
        X_train_raw.append(observed)
        y_train_deltas.append(delta)
        stationary_context.append(scenario_stationary)
    
    stationary_context = np.array(stationary_context)


    X_train = np.array(X_train_raw)
    y_train = np.array(y_train_deltas)

    print(f"Training on {X_train.shape[0]} valid sequences.")
    print(f"Input shape: {X_train.shape}, Delta Output shape: {y_train.shape}")

    # --- Normalize Input and Output ---
    X_mean = X_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 6)
    X_std = X_train.std(axis=(0, 1), keepdims=True) + 1e-8

    y_mean = y_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 2)
    y_std = y_train.std(axis=(0, 1), keepdims=True) + 1e-8

    X_train = (X_train - X_mean) / X_std
    y_train = (y_train - y_mean) / y_std

    stationary_context = np.array(stationary_context)  # Shape: (n_samples, max_stationary, 2)

    # Normalize context - same approach as your other data
    context_mean = stationary_context.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 2)
    context_std = stationary_context.std(axis=(0, 1), keepdims=True) + 1e-8
    
    stationary_context = (stationary_context - context_mean) / context_std

    model = create_lstm_encoder_decoder(
        input_dim=X_train.shape[-1],
        output_dim=2,
        timesteps_in=Tobs,
        timesteps_out=Tpred,
        loss_fn='mse',
        lr=0.001
    )

    phase1_callbacks = [
        LearningRateScheduler(exponential_decay_schedule),
        EarlyStopping(patience=4, restore_best_weights=True, monitor='val_loss'),
        LRThresholdCallback(threshold=9e-5)
    ]

    print("\n--- Phase 1: Training ---")
    model.fit(
        [X_train, stationary_context], y_train,  # List of inputs
        epochs=50,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase1_callbacks,
        verbose=1
    )

    print("\n--- Phase 2: Fine-tuning ---")
    model.compile(optimizer=Adam(1e-4), loss='mse', metrics=['mae'])
    phase2_callbacks = [
        LearningRateScheduler(exponential_decay_schedule),
        EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss')
    ]
    model.fit(
        [X_train, stationary_context], y_train,
        epochs=20,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase2_callbacks,
        verbose=1
    )

    # Return model and normalization parameters
    return model, X_mean, X_std, y_mean, y_std, context_mean, context_std


In [13]:
import pickle

def save_model(model, filepath='lstm_1.pkl'):
    """Save model and scaler together in a pickle file"""
    model_json = model.to_json()
    model_weights = model.get_weights()
    data = {
        'model_json': model_json,
        'model_weights': model_weights,
    }
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"Model saved to {filepath}")

def load_model(filepath='lstm_1.pkl'):
    """Load model and scaler from pickle file"""
    with open(filepath, 'rb') as f:
        data = pickle.load(f)

    # Reconstruct model
    model = tf.keras.models.model_from_json(data['model_json'])
    model.set_weights(data['model_weights'])
    model.compile(optimizer='adam', loss='mse')

    return model

In [74]:
# first check that the model can overfit on small data
model, X_mean, X_std, y_mean, y_std, context_mean, context_std  = train_model(train_data[:20], batch_size=20, validation_split=0)

Training on 20 valid sequences.
Input shape: (20, 50, 6), Delta Output shape: (20, 60, 2)

--- Phase 1: Training ---
Epoch 1/50
1/1 [==============================] - 12s 12s/step - loss: 1.0039 - mae: 0.6571 - lr: 0.0010
Epoch 2/50
1/1 [==============================] - 0s 64ms/step - loss: 0.9688 - mae: 0.6372 - lr: 0.0010
Epoch 3/50
1/1 [==============================] - 0s 66ms/step - loss: 0.7024 - mae: 0.6291 - lr: 0.0010
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 1.2292 - mae: 0.8683 - lr: 0.0010
Epoch 5/50
1/1 [==============================] - 0s 61ms/step - loss: 0.9423 - mae: 0.6617 - lr: 0.0010
Learning rate update: 0.0009000000427477062
Epoch 6/50
1/1 [==============================] - 0s 59ms/step - loss: 0.8056 - mae: 0.5824 - lr: 9.0000e-04
Epoch 7/50
1/1 [==============================] - 0s 56ms/step - loss: 0.8032 - mae: 0.6789 - lr: 9.0000e-04
Epoch 8/50
1/1 [==============================] - 0s 57ms/step - loss: 0.7652 - mae: 0.7072 - lr

In [14]:
def plot_mae_by_timestep(y_true, y_pred):
    """
    Visualize MAE across timesteps in the prediction horizon.

    Args:
        y_true (np.ndarray): shape (N, Tpred, 2)
        y_pred (np.ndarray): shape (N, Tpred, 2)
    """
    mae_per_timestep = np.mean(np.abs(y_true - y_pred), axis=(0, 2))  # shape (Tpred,)

    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    plt.plot(mae_per_timestep, label='MAE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('MAE (meters)')
    plt.title('Mean Absolute Error Over Prediction Horizon')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [15]:
def reconstruct_absolute_positions(pred_deltas, last_observed_positions):
    """
    Reconstruct absolute predicted positions by adding deltas to the last observed position.

    Args:
        pred_deltas: np.ndarray of shape (N, Tpred, 2)
        last_observed_positions: np.ndarray of shape (N, 2)

    Returns:
        np.ndarray of shape (N, Tpred, 2)
    """
    return last_observed_positions[:, None, :] + np.cumsum(pred_deltas, axis=1)



def forecast_positions(scenario_data, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std, 
                      context_mean, context_std, max_stationary=30):
    """
    Use normalized LSTM model to forecast future deltas and reconstruct absolute positions.
    Args:
        scenario_data (numpy.ndarray): Shape (agents, time_steps, dimensions)
        Tobs (int): Number of observed time steps
        Tpred (int): Number of future time steps to predict
        model (Model): Trained LSTM model that predicts normalized deltas
        X_mean, X_std: Normalization stats for input
        y_mean, y_std: Normalization stats for output
        context_mean, context_std: Normalization stats for context
        max_stationary (int): Maximum number of stationary agents in context
    Returns:
        numpy.ndarray: Predicted absolute positions of shape (agents, Tpred, 2)
    """
    agents, _, _ = scenario_data.shape
    predicted_positions = np.zeros((agents, Tpred, 2))
    pred_deltas_all = []
    
    for agent_idx in range(agents):
        agent_data = scenario_data[agent_idx, :Tobs, :]  # shape (Tobs, 6)
        
        # Skip if fully padded
        if np.all(agent_data == 0):
            continue
            
        # Extract stationary context for this agent
        agent_last_pos = agent_data[-1, :2]
        scenario_stationary = []
        
        for other_idx in range(agents):
            if other_idx == agent_idx:  # Skip self
                continue
                
            other_data = scenario_data[other_idx, :Tobs, :]
            if np.all(other_data == 0):
                continue
                
            # Check if other agent is stationary
            relative_pos = other_data[-1, :2] - agent_last_pos
            scenario_stationary.append(relative_pos)
        
        # Pad or limit stationary agents
        if len(scenario_stationary) > max_stationary:
            scenario_stationary = scenario_stationary[:max_stationary]
        else:
            while len(scenario_stationary) < max_stationary:
                scenario_stationary.append([0.0, 0.0])
        
        # Prepare inputs
        X_pred = np.expand_dims(agent_data, axis=0)  # shape (1, Tobs, 6)
        X_pred_norm = (X_pred - X_mean) / X_std
        
        context_pred = np.expand_dims(scenario_stationary, axis=0)  # shape (1, max_stationary, 2)
        context_pred_norm = (context_pred - context_mean) / context_std
        
        # Predict normalized deltas with both inputs
        pred_deltas_norm = model.predict([X_pred_norm, context_pred_norm], verbose=0)  # shape (1, Tpred, 2)
        
        # Denormalize deltas
        pred_deltas = pred_deltas_norm * y_std + y_mean
        pred_deltas_all.append(pred_deltas[0])
        
        # Reconstruct absolute positions
        last_pos = agent_data[Tobs - 1, :2]  # shape (2,)
        abs_positions = reconstruct_absolute_positions(
            pred_deltas=pred_deltas,
            last_observed_positions=np.expand_dims(last_pos, axis=0)
        )[0]
        
        predicted_positions[agent_idx] = abs_positions
    
    return predicted_positions


In [16]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def make_gif(data_matrix1, data_matrix2, name='comparison'):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    cmap1 = plt.cm.get_cmap('viridis', 50)
    cmap2 = plt.cm.get_cmap('plasma', 50)

    assert data_matrix1.shape[1] == data_matrix2.shape[1], "Both matrices must have same number of timesteps"
    timesteps = data_matrix1.shape[1]

    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    ax1, ax2 = axes

    def update(frame):
        for ax in axes:
            ax.clear()

        for i in range(data_matrix1.shape[0]):
            for (data_matrix, ax, cmap) in [(data_matrix1, ax1, cmap1), (data_matrix2, ax2, cmap2)]:
                x = data_matrix[i, frame, 0]
                y = data_matrix[i, frame, 1]
                if x != 0 and y != 0:
                    xs = data_matrix[i, :frame+1, 0]
                    ys = data_matrix[i, :frame+1, 1]
                    mask = (xs != 0) & (ys != 0)
                    xs = xs[mask]
                    ys = ys[mask]
                    if len(xs) > 0 and len(ys) > 0:
                        color = cmap(i)
                        ax.plot(xs, ys, alpha=0.9, color=color)
                        ax.scatter(x, y, s=80, color=color)

        # Plot ego vehicle (index 0) on both
        ax1.plot(data_matrix1[0, :frame, 0], data_matrix1[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax1.scatter(data_matrix1[0, frame, 0], data_matrix1[0, frame, 1], s=80, color='tab:orange')
        ax1.set_title('Prediction')

        ax2.plot(data_matrix2[0, :frame, 0], data_matrix2[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax2.scatter(data_matrix2[0, frame, 0], data_matrix2[0, frame, 1], s=80, color='tab:orange')
        ax2.set_title('Actual')

        for ax, data_matrix in zip(axes, [data_matrix1, data_matrix2]):
            ax.set_xlim(data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].min() - 10,
                        data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].max() + 10)
            ax.set_ylim(data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].min() - 10,
                        data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].max() + 10)
            ax.legend()
            ax.set_xlabel('X')
            ax.set_ylabel('Y')

        # Compute MSE over non-zero entries up to current frame
        mask = (data_matrix2[:, :frame+1, :] != 0) & (data_matrix1[:, :frame+1, :] != 0)
        mse = np.mean((data_matrix1[:, :frame+1, :][mask] - data_matrix2[:, :frame+1, :][mask]) ** 2)

        fig.suptitle(f"Timestep {frame} - MSE: {mse:.4f}", fontsize=16)
        return ax1.collections + ax1.lines + ax2.collections + ax2.lines

    anim = animation.FuncAnimation(fig, update, frames=list(range(0, timesteps, 3)), interval=100, blit=True)
    anim.save(f'trajectory_visualization_{name}.gif', writer='pillow')
    plt.close()


In [17]:
# visualize prediction

# model = load_model()

# Parameters
Tobs = 50
Tpred = 60

data = train_data[6000]

# Select a test scenario (can use any valid index)
test_scenario = data.copy()  # shape (agents, time_steps, features)

# Forecast future positions
predicted_positions = forecast_positions(test_scenario, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std, context_mean, context_std)

# Create combined matrix of past observed + predicted for ego agent (agent 0)
ego_past = test_scenario[0, :Tobs, :2]               # shape (Tobs, 2)
ego_future = predicted_positions[0]                  # shape (Tpred, 2)
ego_full = np.concatenate([ego_past, ego_future], axis=0)  # shape (Tobs + Tpred, 2)

# Create updated scenario with predicted ego and original others
updated_scenario = test_scenario.copy()
updated_scenario[0, :Tobs+Tpred, :2] = ego_full  # Replace ego trajectory

# Visualize
make_gif(updated_scenario, data, name='lstm1')


NameError: name 'model' is not defined

In [ ]:
# Train the model
model, X_mean, X_std, y_mean, y_std, context_mean, context_std = train_model(train_data)

# Save the model
save_model(model)

Training on 10000 valid sequences.
Input shape: (10000, 50, 6), Delta Output shape: (10000, 60, 2)


2025-05-30 00:54:08.272017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10534 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:8a:00.0, compute capability: 6.1



--- Phase 1: Training ---
Epoch 1/50


2025-05-30 00:54:20.062082: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


250/250 [==============================] - 29s 77ms/step - loss: 0.3199 - mae: 0.3827 - val_loss: 0.1791 - val_mae: 0.2816 - lr: 0.0010
Epoch 2/50
250/250 [==============================] - 16s 63ms/step - loss: 0.1602 - mae: 0.2698 - val_loss: 0.1348 - val_mae: 0.2484 - lr: 0.0010
Epoch 3/50
250/250 [==============================] - 16s 63ms/step - loss: 0.1263 - mae: 0.2345 - val_loss: 0.1167 - val_mae: 0.2154 - lr: 0.0010
Epoch 4/50
250/250 [==============================] - 16s 63ms/step - loss: 0.1209 - mae: 0.2280 - val_loss: 0.1166 - val_mae: 0.2143 - lr: 0.0010
Epoch 5/50
250/250 [==============================] - 16s 65ms/step - loss: 0.1100 - mae: 0.2152 - val_loss: 0.1149 - val_mae: 0.2142 - lr: 0.0010
Learning rate update: 0.0009000000427477062
Epoch 6/50
250/250 [==============================] - 16s 65ms/step - loss: 0.1042 - mae: 0.2085 - val_loss: 0.1101 - val_mae: 0.2205 - lr: 9.0000e-04
Epoch 7/50
250/250 [==============================] - 16s 64ms/step - loss: 0.098

In [83]:
from sklearn.metrics import mean_squared_error


def evaluate_mse(train_data, model, Tobs=50, Tpred=60):
    """
    Computes LSTM prediction for ego agent and evaluates MSE with progress reporting.
    """
    N = train_data.shape[0]
    mse_list = []
    valid_scenarios = 0

    print(f"Evaluating {N} scenarios...")

    # Progress reporting variables
    report_interval = max(1, N // 10)  # Report at 10% intervals

    for i in range(N):
        # Progress reporting
        if i % report_interval == 0 or i == N-1:
            print(f"Processing scenario {i+1}/{N} ({(i+1)/N*100:.1f}%)")

        scenario_data = train_data[i]
        ego_agent_data = scenario_data[0]
        ground_truth = ego_agent_data[Tobs:Tobs+Tpred, :2]

        # Skip if ground truth contains all zeros (padded)
        if np.all(ground_truth == 0):
            continue

        valid_scenarios += 1

        # Forecast future positions
        predicted_positions = forecast_positions(
            ego_agent_data[np.newaxis, :, :],
            Tobs, Tpred, model, X_mean, X_std, y_mean, y_std, context_mean, context_std
        )

        # Compute MSE
        mse = mean_squared_error(ground_truth, predicted_positions[0])
        mse_list.append(mse)

        # Occasional MSE reporting
        if i % report_interval == 0:
            print(f"  Current scenario MSE: {mse:.4f}")

    # Final results
    if mse_list:
        overall_mse = np.mean(mse_list)
        print(f"Evaluation complete: {valid_scenarios} valid scenarios")
        print(f"Mean Squared Error (MSE): {overall_mse:.4f}")
        print(f"Min MSE: {np.min(mse_list):.4f}, Max MSE: {np.max(mse_list):.4f}")
        return overall_mse
    else:
        print("No valid scenarios for evaluation.")
        return None

In [84]:
# Evaluate on training data
evaluate_mse(train_data, model)

Evaluating 10000 scenarios...
Processing scenario 1/10000 (0.0%)
  Current scenario MSE: 0.6715
Processing scenario 1001/10000 (10.0%)
  Current scenario MSE: 6.1291
Processing scenario 2001/10000 (20.0%)
  Current scenario MSE: 0.5003
Processing scenario 3001/10000 (30.0%)
  Current scenario MSE: 0.0499
Processing scenario 4001/10000 (40.0%)
  Current scenario MSE: 0.4941
Processing scenario 5001/10000 (50.0%)
  Current scenario MSE: 14.7437
Processing scenario 6001/10000 (60.0%)
  Current scenario MSE: 88.2834
Processing scenario 7001/10000 (70.0%)
  Current scenario MSE: 11.7321


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

def generate_submission(data, output_csv, Tobs=50, Tpred=60):
    """
    Applies forecasting and generates a submission CSV with format:
    index,x,y where index is auto-generated and matches submission key.

    Args:
        data (np.ndarray): Test data of shape (num_scenarios, 50, 50, 6).
        output_csv (str): Output CSV file path.
        Tobs (int): Observed time steps (default 50).
        Tpred (int): Prediction time steps (default 60).
    """

    predictions = []

    for i in range(data.shape[0]):
        scenario_data = data[i]            # Shape: (50, 50, 6)
        ego_agent_data = scenario_data[0]  # Shape: (50, 6)

        # Predict future positions for the ego agent
        predicted_positions = forecast_positions(
            ego_agent_data[np.newaxis, :, :], Tobs, Tpred, model, X_mean, X_std, y_mean, y_std
        )  # Shape: (1, 60, 2)

        # Append 60 predictions (x, y) for this scenario
        predictions.extend(predicted_positions[0])  # Shape: (60, 2)

    # Create DataFrame without explicit ID
    submission_df = pd.DataFrame(predictions, columns=["x", "y"])
    submission_df.index.name = 'index'  # Match Kaggle format

    # Save CSV with index
    submission_df.to_csv(output_csv)
    print(f"Submission file '{output_csv}' saved with shape {submission_df.shape}")

generate_submission(test_data, 'lstm_submission.csv')